<a href="https://colab.research.google.com/github/jswooo/HuggingFace/blob/main/2_3(Tokenizers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]

In [1]:
'''
model이 이해하기 위해서는 raw data(text)를 숫자로 바꿔줄 필요가 있음 => 이 역할을 tokenizer가 수행 (model에 잘 맞고, 가능하다면 가장 작은 representation)
tokenizer 알고리즘은 3가지가 존재. 
1. word-based
2. character-based
3. subword-based
'''

'\nmodel이 이해하기 위해서는 raw data(text)를 숫자로 바꿔줄 필요가 있음 => 이 역할을 tokenizer가 수행\ntokenizer 알고리즘은 3가지가 존재. \n1. word-based\n2. character-based\n3. subword-based\n'

### word-based tokenization

In [ ]:
'''
마침표나 단어 별로 나눠서 토큰화 

장점: 규칙이 매우 쉬움 
단점: 
1) dog나 dogs와 같은 유사 단어의 유사성을 보여주지 못함. UNK token(모르는 단어)들 모두 의미가 같게 부여. => 의미를 잃어버릴수도
2) 모든 단어에 대한 ID를 부여해야 하기에, 모델이나 토큰들의 크기가 너무 커짐.
'''

In [1]:
tokenized_text = "jim Henson was a puppeteer".split() # 예시단어 
print(tokenized_text)

['jim', 'Henson', 'was', 'a', 'puppeteer']


### character-based tokenization

In [ ]:
'''
character로 토큰화 

장점: 
1) 단어 토큰화에 비해 크기가 작아짐. 모든 단어를 토큰화 하려면 170,000개가 필요하지만, character 기반은 256개면 됌 
2) 모든 단어들은 character로 이뤄져있기에, OOV가 매우 적어짐

단점: 
1) 단어에 비해서 토큰이 나타내는 의미가 작아짐 (단, 언어에 따라서 달라지기도 함)
2) corpus를 나타내기 위해서 매우 많은 토큰이 필요함. (word-based로는 1개인데, char~ based는 여러개)
'''

### subword-based tokenization

In [ ]:
'''
빈번하게 사용되는 단어는 작은 subword로 나누지 않고, 그렇지 않은 단어는 의미있는 subword로 분해 (word-based와 character-based의 사이)

e.g. annoyingly => annoying + ly / Let's do tokenization! => let's + do + token + ization + ! 

각 모델에 대한 tokenization 방법들 
1) wordpiece => BERT, DistilBERT
2) Unigram => XLNet, ALBERT
3) Byte-pair Encoding => GPT-2, RoBERTa 
'''

### Loading and Saving 

In [4]:
from transformers import BertTokenizer 

# loading 
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [5]:
print(tokenizer)

BertTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


In [6]:
from transformers import AutoTokenizer 

# Automodel과 마찬가지로 해당 모델에 가장 적절한 tokenizer class를 불러와 줌 
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [7]:
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)


In [8]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
# saving 
tokenizer.save_pretrained("directory_on_my_computer") # directory_on_my_computer라는 폴더 내에 토크아니저 관련 파일들 저장

('directory_on_my_computer/tokenizer_config.json',
 'directory_on_my_computer/special_tokens_map.json',
 'directory_on_my_computer/vocab.txt',
 'directory_on_my_computer/added_tokens.json',
 'directory_on_my_computer/tokenizer.json')

### encoding 

In [ ]:
'''
text => number로 바꾸는 과정 

step 1. text를 토큰으로 바꾸는 과정(토큰화 방법이 매우 다양하기 때문에 모델이 사전학습 할때 사용했던 방식과 동일한 방법을 사용해야 해야 함)
step 2. 토큰을 숫자로 바꾸는 과정(이때 토큰들은 vocabulary를 가지고 있고, 마찬가지로 모델이 사전학습 될때 사용한 voca와 동일한 것을 사용해야 함)
'''

In [2]:
# 토큰화 과정 

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") # tokenizer 선정 
sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens) #subword 토큰화임 (Trans와 Former로 나눠짐을 통해 알 수 있음)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


In [3]:
# input id로 변환 

ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids) #이제 이 결과 값을 텐서로 변경하여 model의 input으로 사용 

[7993, 170, 13809, 23763, 2443, 1110, 3014]


### Decoding

In [4]:
## decoding은 최종적으로 word를 반환 
## decoding은 숫자를 token으로 반환할 뿐만 아니라, 읽을 수 있는 문장으로도 바꿔줌 => 새로운 text를 예측하는 text generator, summarization, translation 등에서 유리) 
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])

print(decoded_string)

Using a transformer network is simple
